In [1]:
import json
import random
import pandas as pd

# Load

In [2]:
def read_ultra_data(file_path):
    with open(file_path, 'r', encoding="utf-8") as file:
        session_list = file.readlines()
    return session_list
d = read_ultra_data(r'E:\ra\w4-10\code\reallm\corpus\dolly\databricks-dolly-15k.jsonl')

In [3]:
d_lst = []
for session_str in d:
    d_lst.append(eval(session_str))

In [4]:
len(d_lst)

15011

# Sample

In [7]:
def stratified_sampling(data, key, sample_size, random_seed):
    random.seed(random_seed)  
    
    count_dict = {}
    for item in data:
        value = item[key]
        count_dict[value] = count_dict.get(value, 0) + 1

    total_samples = len(data)
    proportions = {k: v / total_samples for k, v in count_dict.items()}
    
    sample_sizes = {k: round(v * sample_size) for k, v in proportions.items()}
    
    sampled_data = []
    for value, sample_size in sample_sizes.items():
        group = [item for item in data if item[key] == value]
        if len(group) < sample_size:
            sampled_data.extend(group)
        elif len(group) == sample_size:
            sampled_data.extend(group)
        else:
            sampled_data.extend(random.sample(group, sample_size))
    
    return sampled_data

sampled_data = stratified_sampling(d_lst, 'category', 10000, 125)

In [8]:
sample_df = pd.DataFrame(sampled_data)
sample_df['category'].value_counts()

category
open_qa                   2493
general_qa                1460
classification            1423
closed_qa                 1181
brainstorming             1176
information_extraction    1003
summarization              791
creative_writing           472
Name: count, dtype: int64

In [9]:
# check
sampled_data[0]
# 'instruction': 'Given these paragraphs about the founding of Tesla, who else part from Elon Musk and J. B. Straubel are co-founders of Tesla?',
#  'context': 'The company was inc

{'instruction': 'Given these paragraphs about the founding of Tesla, who else part from Elon Musk and J. B. Straubel are co-founders of Tesla?',
 'context': 'The company was incorporated as Tesla Motors, Inc. on July 1, 2003, by Martin Eberhard and Marc Tarpenning. Eberhard and Tarpenning served as CEO and CFO, respectively. Eberhard said he wanted to build "a car manufacturer that is also a technology company", with its core technologies as "the battery, the computer software, and the proprietary motor".\n\nIan Wright was Tesla\'s third employee, joining a few months later. In February 2004, the company raised $7.5 million in series A funding, including $6.5 million from Elon Musk, who had received $100 million from the sale of his interest in PayPal two years earlier. Musk became the chairman of the board of directors and the largest shareholder of Tesla. J. B. Straubel joined Tesla in May 2004 as chief technical officer.\n\nA lawsuit settlement agreed to by Eberhard and Tesla in Sep

In [10]:
len(sampled_data)

9999

In [12]:
# Supplement one session
for session_dict in d_lst:
    if session_dict['category'] == 'open_qa':
        if str(session_dict) in str(sampled_data):
            continue
        else:
            sampled_data.append(session_dict)
            break

In [13]:
len(sampled_data)

10000

In [14]:
sample_df = pd.DataFrame(sampled_data)
sample_df['category'].value_counts()

category
open_qa                   2494
general_qa                1460
classification            1423
closed_qa                 1181
brainstorming             1176
information_extraction    1003
summarization              791
creative_writing           472
Name: count, dtype: int64

# Post-processing

In [15]:
sampled_data[0]

{'instruction': 'Given these paragraphs about the founding of Tesla, who else part from Elon Musk and J. B. Straubel are co-founders of Tesla?',
 'context': 'The company was incorporated as Tesla Motors, Inc. on July 1, 2003, by Martin Eberhard and Marc Tarpenning. Eberhard and Tarpenning served as CEO and CFO, respectively. Eberhard said he wanted to build "a car manufacturer that is also a technology company", with its core technologies as "the battery, the computer software, and the proprietary motor".\n\nIan Wright was Tesla\'s third employee, joining a few months later. In February 2004, the company raised $7.5 million in series A funding, including $6.5 million from Elon Musk, who had received $100 million from the sale of his interest in PayPal two years earlier. Musk became the chairman of the board of directors and the largest shareholder of Tesla. J. B. Straubel joined Tesla in May 2004 as chief technical officer.\n\nA lawsuit settlement agreed to by Eberhard and Tesla in Sep

In [16]:
merged_q_lst = []
for i, session_dict in enumerate(sampled_data):
    new_dict = dict()
    new_dict['id'] = i
    if session_dict['category'] == 'closed_qa':
        q = 'This is a Closed Question-Answering task.\n' + session_dict['instruction']  + '\n' + session_dict['context']  
    elif session_dict['category'] == 'summarization':
        q = 'This is a Summarization task.\n' + session_dict['instruction'] + '\n' + session_dict['context']
    elif session_dict['category'] == 'information_extraction':
        q = 'This is a Information Extraction task.\n' + session_dict['instruction'] + '\n' + session_dict['context']
    else:
        q = session_dict['instruction']
    new_dict['question'] = q
    merged_q_lst.append(new_dict)
assert len(sampled_data) == len(merged_q_lst)

In [17]:
merged_q_lst[0]

{'id': 0,
 'question': 'This is a Closed Question-Answering task.\nGiven these paragraphs about the founding of Tesla, who else part from Elon Musk and J. B. Straubel are co-founders of Tesla?\nThe company was incorporated as Tesla Motors, Inc. on July 1, 2003, by Martin Eberhard and Marc Tarpenning. Eberhard and Tarpenning served as CEO and CFO, respectively. Eberhard said he wanted to build "a car manufacturer that is also a technology company", with its core technologies as "the battery, the computer software, and the proprietary motor".\n\nIan Wright was Tesla\'s third employee, joining a few months later. In February 2004, the company raised $7.5 million in series A funding, including $6.5 million from Elon Musk, who had received $100 million from the sale of his interest in PayPal two years earlier. Musk became the chairman of the board of directors and the largest shareholder of Tesla. J. B. Straubel joined Tesla in May 2004 as chief technical officer.\n\nA lawsuit settlement ag

# Output

In [18]:
for i, session_dict in enumerate(merged_q_lst):
    new_q = session_dict['question'].encode('UTF-8', 'ignore').decode('UTF-8')
    merged_q_lst[i]['question'] = new_q

In [19]:
len(merged_q_lst)

10000

In [20]:
with open('dolly_q_10k.json', "w", encoding="utf-8") as f:
    json.dump(merged_q_lst, f, indent=2, ensure_ascii=False)